# Streaming (Akış Yanıtları)

## 🎯 Bu Kodun Amacı
AI'nın yanıtını tamamlanmasını beklemeden kelime kelime görmek (ChatGPT gibi).

## Ne Yapacağız?
- **stream=True** parametresi ile akış modunu aktif edeceğiz
- Havayolu veri setinin istatistiklerini hazırlayacağız
- AI'dan veri analiz raporu isteyeceğiz
- Yanıt kelime kelime gelecek ve ekrana **anında** yazdırılacak
- `flush=True` ile buffer'ı temizleyip gerçek zamanlı görüntüleme sağlayacağız

## Çalıştırma
Kod çalıştırıldığında AI'nın raporu kelime kelime yazarken izleyebilirsiniz.

In [ ]:
import os
from dotenv import load_dotenv
from groq import Groq
import pandas as pd

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

df = pd.read_csv('airline_passenger_satisfaction.csv')

## 📊 Veri İstatistikleri ve Prompt Hazırlama

**Ne yapıyoruz?**
- Veri hakkında kısa bir özet hazırlıyoruz
- Bu özeti kullanarak AI'dan rapor isteyeceğimiz prompt'u oluşturuyoruz

**stats değişkeni:**
- `len(df)`: Toplam satır sayısı
- `len(df.columns)`: Kolon sayısı  
- `df.select_dtypes(include=['number']).columns.tolist()[:5]`: İlk 5 numerik kolon adı

**prompt:**
- AI'dan veri yapısı, olası analizler ve öneriler içeren kısa bir rapor istiyoruz

In [ ]:
stats = f"""
Toplam Kayıt: {len(df)}
Kolon Sayısı: {len(df.columns)}
Numerik Kolonlar: {df.select_dtypes(include=['number']).columns.tolist()[:5]}
"""

prompt = f"""Bu havayolu veri setini analiz et ve kısa bir rapor yaz:

{stats}

Rapor: veri yapısı, olası analizler ve öneriler içermeli."""

## 🌊 Streaming (Akış) Modunda Yanıt Alma

**Ne yapıyoruz?**
- `stream=True` parametresi ile yanıtı kelime kelime alıyoruz
- ChatGPT'deki gibi gerçek zamanlı yazım efekti

**stream=True:**
- Yanıt tamamlanmadan gelmeye başlar
- Her "chunk" (parça) kelime kelime gelir

**for chunk in stream:**
- Her gelen parçayı döngüyle işliyoruz
- `chunk.choices[0].delta.content`: O andaki yeni kelime/kelimeler

**end=''**
- Satır atlamadan yazdır (aynı satırda devam et)

**flush=True:**
- Buffer'ı temizle, anında ekrana yaz
- Olmazsa kelimeler toplanır ve toplu gelir (streaming etkisi olmaz)

**Neden streaming?**
- Kullanıcı deneyimi: Bekleme hissi azalır
- Gerçek zamanlı görüntü: Uzun cevaplarda kullanıcı hemen okumaya başlar
- ChatGPT, Claude gibi modern AI arayüzlerinin standardı

In [ ]:
print("ANALIZ RAPORU:\n")

stream = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": prompt}],
    stream=True,
    temperature=0.7,
    max_tokens=500
)

for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end='', flush=True)

print("\n")

ANALIZ RAPORU:

**Havayolu Veri Seti Analiz Raporu**

**Veri Yapısı:**

Toplam 129.880 kayıt ve 24 kolon içeren havayolu veri seti, geniş bir veri yapısına sahiptir. Veri setinde bulunan 24 kolonun içinde 5 tanesi ('ID', 'Age', 'Flight Distance', 'Departure Delay' ve 'Arrival Delay') numerik kolonlardır. Bu, veri setinin büyük kısmının kategorik veya metinsel veri içerdiğini gösterir.

**Olası Analizler:**

1. **Uçuş Gecikme Analizi:** 'Departure Delay' ve 'Arrival Delay' kolonlarını kullanarak, uçuş gecikme oranlarını ve nedenlerini analiz edebiliriz. Bu, havayolu şirketlerinin operasyonel verimliliğini artırmasına yardımcı olabilir.
2. **Yolcu Yaşı ve Uçuş Mesafesi İlişkisi:** 'Age' ve 'Flight Distance' kolonlarını kullanarak, yolcu yaşının uçuş mesafesiyle nasıl ilişkili olduğunu inceleyebiliriz. Bu, havayolu şirketlerinin yolcu profillerini daha iyi anlamalarına yardımcı olabilir.
3. **ID ve Yolcu Bilgileri İlişkisi:** 'ID' kolonunu kullanarak, her bir yolcunun uçuş geçmişini ve te